In [4]:
import pandas as pd
from chanlun import cl
from chanlun import kcharts

### 避免在 Notebook 中不显示，手动加载下依赖的js
###　https://pyecharts.org/#/zh-cn/notebook
from pyecharts.charts import Kline, Line, Bar, Grid, Scatter
from pyecharts.globals import CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB  # 根据使用的环境设置 notebook 类型， NotebookType.JUPYTER_LAB 或 NotebookType.JUPYTER_NOTEBOOK
Kline().load_javascript()
Line().load_javascript()
Bar().load_javascript()
Grid().load_javascript()
Scatter().load_javascript()

## 缠论配置项
* bi_type: 默认值 old；old 老笔 new 新笔 dd 顶底成笔
* no_bi： 默认值 False；False 笔未出现结束分型，不计入笔；  True  笔未出结束分型，但是符合成笔条件，计入笔
* zs_type: 默认值 dn； dn 表示根据线段，在线段内查找中枢； bl 表示遍历并查找所有符合中枢条件的中枢
* zs_qj: 默认值 hl； hl 中枢区间使用实际的高低点； dd 中枢区间使用线的顶底端点
* fx_baohan: 默认值 False；False 不允许有分型包含条件成笔； True 允许有分型包含成笔
* idx_macd_fast: 默认值 12；MACD 指标快线参数
* idx_macd_slow: 默认值 26；MACD 指标慢线参数
* idx_macd_signal: 默认值 9；MACD 移动平均周期
* idx_boll_period：默认值 20；布林线参数
* idx_ma_period：默认值 5；均线参数

## 画图配置
* show_bi_zs：默认值 True；是否显示笔中枢
* show_xd_zs：默认值 True；是否显示线段中枢
* show_bi_mmd：默认值 True；是否显示笔的买卖点
* show_xd_mmd：默认值 True；是否显示线段的买卖点
* show_bi_bc：默认值 True；是否显示笔背驰
* show_xd_bc：默认值 True；是否显示线段背驰
* show_ma：默认值 True；是否显示均线
* show_boll：默认值 True；是否显示布林线

In [ ]:
# 获取 行情K线数据
code = 'SH.688122'
frequency = '30m'
# 缠论配置
cl_config = {'bi_type': 'old', 'no_bi': False, 'zs_type': 'dn', 'zs_qj': 'hl', 'fx_baohan': True}
# 缠论画图配置
chart_config = {'show_bi_zs': True, 'show_ma': False}
klines = pd.read_csv('./data/688122.csv')
klines


In [6]:
# 依据 K 线数据，计算缠论数据
cl_data = cl.CL(code, frequency, cl_config).process_klines(klines)
chart = kcharts.render_charts('%s - %s' % (code, frequency), cl_data, config=chart_config)

In [ ]:
# 展示图标
chart

In [8]:
# 获取最后一笔
bi = cl_data.bis[-1]
print('Last Bi : ', bi)
# 获取最后一个中枢
zs = cl_data.bi_zss[-1]
print('Last Zs : ', zs)

Last Bi :  index: 44 type: up FX: (2022-02-22 11:00:00 - 2022-02-28 10:30:00) high: 86.63 low: 73.7 done: True
Last Zs :  index: 8 zs_type: bi level: 0 FX: (2022-01-25 10:00:00-2022-02-16 10:00:00) type: zd zg: 81.17 zd: 74.23 gg: 83.76 dd: 73.07 done: True


In [11]:
# 判断笔是否背驰，是否有买卖点
bi = cl_data.bis[-11]
print('BI MMDS : ', bi.line_mmds())
print('BI BC : ', [_bc.type for _bc in bi.bcs if _bc.bc])
print('BI 笔背驰：', bi.bc_exists(['bi']))
print('BI 盘整背驰：', bi.bc_exists(['pz']))
print('BI 趋势背驰：', bi.bc_exists(['qs']))

BI MMDS :  ['l3sell']
BI BC :  []
BI 笔背驰： False
BI 盘整背驰： False
BI 趋势背驰： False


In [12]:
# 买卖点信息
mmd = bi.mmds[0]
print(mmd)
print('买卖点对应的中枢：', mmd.zs)

MMD: l3sell ZS: index: 6 zs_type: bi level: 0 FX: (2021-12-08 11:30:00-2021-12-28 11:00:00) type: up zg: 90.5 zd: 89.23 gg: 93.99 dd: 86.35 done: True
买卖点对应的中枢： index: 6 zs_type: bi level: 0 FX: (2021-12-08 11:30:00-2021-12-28 11:00:00) type: up zg: 90.5 zd: 89.23 gg: 93.99 dd: 86.35 done: True


## Html 页面展示

### 服务端代码
    return HttpResponse(kcharts.render_charts('标题', cl_datas))

### 页面 Ajax 接收并展示
    var chart = echarts.init(document.getElementById('chart_id'), 'drak', {renderer: 'canvas'});
    $.ajax({
        type: "GET",
        url: "/后端请求地址",
        success: function (result) {
            var res = (new Function("return " + result))();
            chart.setOption(res);
        }
    });